In [1]:
import pandas as pd
import random
import numpy as np

In [2]:
# Set random seed for reproducibility
random.seed(60)
np.random.seed(60)

In [3]:
# Real data
towns_data = {
    'Service_town': ['Gbi - Kledzo', 'Gbi - Wegbe', 'Godenu', 'Gbi - Avege', 'Ve-Dator',
                     'Ve- Kobenu', 'Golokwati', 'Afadzo South', 'Ve -Gbodome', 'Wuinta',
                     'Logba', 'Fume', 'Fodome', 'Liati', 'Zimugaziwo snake Village',
                     'Wli', 'Leklebi', 'Gbledi', 'Agome yo', 'Santrokofi', 'Akpafu',
                     'Lolobi', 'Likpe'],
    'distance_in_km': [4.9, 6.2, 19.2, 10.2, 14.3, 13.7, 21.3, 38.4, 25.4, 30.7,
                       38.3, 38.3, 0.65, 20, 16.5, 22.8, 23.5, 1.3, 15.9, 7.5,
                       11.9, 10.1, 17.2],
    'travel_time_in_hours': [0.267, 0.333, 0.5, 0.467, 0.6, 0.583, 0.5, 1.017, 0.667, 0.783,
                             1, 0.917, 0.65, 0.55, 0.467, 0.483, 0.633, 0.05, 0.517, 0.333,
                             0.417, 0.35, 0.467]

}

# Create DataFrame
df_towns = pd.DataFrame(towns_data)
df_towns

,Service_town,distance_in_km,travel_time_in_hours
0,Gbi - Kledzo,4.90,0.267
1,Gbi - Wegbe,6.20,0.333
2,Godenu,19.20,0.500
3,Gbi - Avege,10.20,0.467
4,Ve-Dator,14.30,0.600
5,Ve- Kobenu,13.70,0.583
6,Golokwati,21.30,0.500
7,Afadzo South,38.40,1.017
8,Ve -Gbodome,25.40,0.667
9,Wuinta,30.70,0.783


In [4]:
df_towns["distance_in_km"].max()

38.4

In [10]:
# Classify zones (Near: 0-20km, Far: 20-40km)
df_towns['zone'] = df_towns['distance_in_km'].apply(lambda x: 'Near' if x <= 20 else 'Far')